#### https://medium.com/@adamaulia/object-tracking-using-opencv-python-windows-616fb23da720

## Setup

In [3]:
import numpy as np
import cv2
import sys

In [6]:
VID_PATH = 'video2.avi'

## Extracting boxes using BGSubtraction

In [7]:
"""Will give boxes for each frame and simultaneously extract background"""

cap  = cv2.VideoCapture(VID_PATH)
fps = int(cap.get(cv2.CAP_PROP_FPS))
fgbg = cv2.createBackgroundSubtractorKNN()

ret, frame = cap.read()
all_conts = []

avg2 = np.float32(frame) #BG-Ext

while ret:
    
    #Background extraction
    try:
        cv2.accumulateWeighted(frame, avg2, 0.01)
    except:
        break
    #if ret is true than no error with cap.isOpened
    ret, frame = cap.read()
    
    if ret==True:
        #apply background substraction
        fgmask = fgbg.apply(frame)  
        
        #apply contours on foreground
        (contours, hierarchy) = cv2.findContours(fgmask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        
        contours = np.array([np.array(cv2.boundingRect(c)) for c in contours if cv2.contourArea(c) >= 500])
        all_conts.append(contours)
        for c in contours:
            
            #get bounding box from countour
            (x, y, w, h) = c
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
#         cv2.imshow('rgb', frame)
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()
background = cv2.convertScaleAbs(avg2)

## Object tracking 

In [8]:
def distance(p1, p2):
    return np.linalg.norm(p1 - p2, axis=1)

def get_nearest(p1, points):
    """returns index of the point in *points* that is closest to p1"""
    return np.argmin(distance(p1, points))

In [9]:
class box:
    def __init__(self, coords, time):
        self.coords = coords #coordinates
        self.time   = time #nth frame/time
        
class moving_obj:
    def __init__(self, starting_box):
        self.boxes = [starting_box]
    
    def add_box(self, box):
        self.boxes.append(box)
    
    def last_coords(self):
        return self.boxes[-1].coords
    
    def age(self, curr_time):
        last_time = self.boxes[-1].time
        return curr_time - last_time    

In [10]:
"""Will associate boxes into objects"""
#old - boxes in the previous frame
#new - boxes in the current frame

THRESHOLD = 10
moving_objs = []

for curr_time, new_boxes in enumerate(all_conts): #iterating over frames
    if len(new_boxes) != 0: #if not empty
        new_assocs = [None]*len(new_boxes) #all new boxes initially arent associated with any moving_objs
        obj_coords = np.array([obj.last_coords() for obj in moving_objs if obj.age(curr_time)<THRESHOLD])
        unexp_idx = -1 #index of unexpired obj in moving_objs
        for obj_idx, obj in enumerate(moving_objs):
            if obj.age(curr_time) < THRESHOLD: #checking only unexpired objects
                unexp_idx += 1
                nearest_new = get_nearest(obj.last_coords(), new_boxes) #nearest box to obj
                nearest_obj = get_nearest(new_boxes[nearest_new], obj_coords) #nearest obj to box

                if nearest_obj==unexp_idx: #both closest to each-other
                    #associate
                    new_assocs[nearest_new] = obj_idx
    
    
    for new_idx, new_coords in enumerate(new_boxes):
        new_assoc = new_assocs[new_idx]
        new_box = box(new_coords, curr_time)

        if new_assoc is not None: 
            #associate new box to moving_obj
            moving_objs[new_assoc].add_box(new_box)
        else: 
            #add a fresh, new moving_obj to moving_objs
            new_moving_obj = moving_obj(new_box)
            moving_objs.append(new_moving_obj)

In [11]:
#Removing objects that occur for a very small duration
MIN_SECONDS = 2 #seconds
MIN_FRAMES = MIN_SECONDS*fps

moving_objs = [obj for obj in moving_objs if (obj.boxes[-1].time-obj.boxes[0].time)>MIN_FRAMES]

## Overlaying moving objects on background

In [12]:
def cut(image, coords):
    (x, y, w, h) = coords
    return image[y:y+h,x:x+w]

In [13]:
def overlay(frame, image, coords):
    (x, y, w, h) = coords
    frame[y:y+h,x:x+w] = cut(image, coords)

In [14]:
import time as tm
def sec2HMS(seconds):
    return tm.strftime('%M:%S', tm.gmtime(seconds))

def frame2HMS(n_frame, fps):
    return sec2HMS(float(n_frame)/float(fps))

In [15]:
max_orig_len = max(obj.boxes[-1].time for obj in moving_objs)
max_duration = max((obj.boxes[-1].time - obj.boxes[0].time) for obj in moving_objs)
#max_duration of a moving_obj. This is taken as the duration of the final summary
start_times = [obj.boxes[0].time for obj in moving_objs]

INTERVAL = 10 #seconds
DIVISIONS = int(max_orig_len/(INTERVAL))
GAP_BW_DIVISIONS = 2 #seconds


final_video  = [background.copy() for _ in range(max_duration+int(DIVISIONS*GAP_BW_DIVISIONS)+10)] #initializing frames of final video

In [16]:
"""Crop moving objects from main video and overlay them on the backgroung"""
cap  = cv2.VideoCapture(VID_PATH)
# fgbg = cv2.createBackgroundSubtractorMOG2()

ret, frame = cap.read() #original video
# all_conts = []

all_texts = []

vid_time = -1
while ret:
    vid_time += 1
    ret, frame = cap.read()
    
    if ret==True:
        
        for obj_idx, mving_obj in enumerate(moving_objs):
            if mving_obj.boxes: #non-empty
                first_box = mving_obj.boxes[0]
                
                if(first_box.time == vid_time):
                    final_time = first_box.time - start_times[obj_idx] + int(int(start_times[obj_idx]/int(INTERVAL*fps))*GAP_BW_DIVISIONS*fps)
                    
                    overlay(final_video[final_time-1], frame, first_box.coords)
                    (x, y, w, h) = first_box.coords
                    
#                     all_texts.append((final_time-1, frame2HMS(first_box.time, fps), (x, y-10))) #Above
                    all_texts.append((final_time-1, frame2HMS(first_box.time, fps), (x+int(w/2), y+int(h/2)))) #Centre
    
                    del(mving_obj.boxes[0])
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()

In [21]:
#annotating moving objects
for (t, text, org) in all_texts:
    cv2.putText(final_video[t], text, org, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (252, 240,3 ), 2)

## Final video

In [22]:
out = cv2.VideoWriter('summary.avi',cv2.VideoWriter_fourcc(*'DIVX'), fps, (background.shape[1],background.shape[0]))

for frame in final_video:
#     cv2.imshow('Video summary',frame)
    out.write(frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
#     tm.sleep(1/30) #TODO: FPS
out.release()
cv2.destroyAllWindows()

In [19]:
cap.release()

In [ ]:
#TODO: Remove overlapping
#mog vs KNN